In [ ]:
# hide
!nvidia-smi

Fri May 28 11:25:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# hide
import sys
if 'google.colab' in sys.modules:
    !pip install -Uqq fastai transformers datasets wandb sentencepiece
    !pip install -q git+git://github.com/aikindergarten/fasthugs.git

In [ ]:
#all_slow

# Machine Translation: fr-en

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, MarianConfig, AutoTokenizer, AutoConfig
from fastai.text.all import *
from fastai.callback.wandb import *

from fasthugs.learner import TransLearner
from fasthugs.data import TransformersTextBlock, TextGetter

## Setup

Let's define main settings for the run in one place:

In [ ]:
model_name = "Helsinki-NLP/opus-mt-fr-en"

max_len = 512
bs = 8
val_bs = bs*2

lr = 2e-5

In [ ]:
df = pd.read_csv('./questions_easy.csv')
df.head()

,en,fr
0,What is light ?,Qu’est-ce que la lumière?
1,Who are we?,Où sommes-nous?
2,Where did we come from?,D'où venons-nous?
3,What would we do without it?,Que ferions-nous sans elle ?
4,"What is the absolute location (latitude and longitude) of Badger, Newfoundland and Labrador?","Quelle sont les coordonnées (latitude et longitude) de Badger, à Terre-Neuve-etLabrador?"


## Dataloaders

In [ ]:
#hide
config = AutoConfig.from_pretrained(model_name)
config

MarianConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      59513
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 59513,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_length": 512,
  "max_position_embeddings": 512,
  "mode

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
@ItemTransform
def untuple1(x):
    return (*x[0], )

In [ ]:
dblock = DataBlock(
    blocks = [TransformersTextBlock(tokenizer=tokenizer, do_targets=True)],
    get_x=TextGetter('fr', 'en'),
    item_tfms=untuple1,
    splitter=RandomSplitter())

In [ ]:
#hide
# dblock.summary(df)

In [ ]:
%%time
bs = 16
dls = dblock.dataloaders(df, bs=bs, val_bs=bs*2, shuffle=True)

CPU times: user 20.8 s, sys: 1.19 s, total: 21.9 s
Wall time: 29.4 s


In [ ]:
#hide
b = dls.one_batch()
b

({'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          ...,
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'),
  'input_ids': tensor([[  277,    34,  1411,  ..., 40236,    54,     0],
          [  315,  4616,    38,  ..., 59513, 59513, 59513],
          [ 2091,  1755,    19,  ..., 59513, 59513, 59513],
          ...,
          [  419,    62,     1,  ..., 59513, 59513, 59513],
          [ 4290,     9,  1477,  ..., 59513, 59513, 59513],
          [ 4717,  1032,   439,  ..., 59513, 59513, 59513]], device='cuda:0'),
  'labels': tensor([[ 1111, 10911,     2,  ...,  1699,    54,     0],
          [35853,     2,    18,  ..., 59513, 59513, 59513],
          [  430,  2357,   165,  ..., 59513, 59513, 59513],
          ...,
          [  430,   235,    61,  ..., 59513, 59513, 59513],
          [  430,  1188,    94,  ..., 59513, 59513, 59513],


In [ ]:
dls.show_batch(max_n=4)

,text,text_
0,"▁Dans un tel▁cas,▁où il sagit d apprécier si un▁nom commercial a un▁fondement▁juridique▁antérieur à▁celui d une▁marque aux▁fins de larticle 16,▁paragraphe 1,▁troisième phrase, de laccord▁ADPIC,▁peut-on▁considérer▁comme▁décisif: i) le▁fait que, dans lÉtat▁où la▁marque est▁enregistrée et sa protection▁réclamée, le▁nom commercial▁ait▁été, du▁moins dans une▁certaine▁mesure,▁connu dans les▁milieux▁professionnels▁int éressés de lÉtat▁concerné▁avant la date à▁laquelle lenregistrement de la▁marque y a▁été▁demandé; ou que, dans les relations▁commerciales▁intéressant lÉtat▁où la▁marque est enregistr ée et sa protection▁réclamée, le▁nom commercial▁ait▁été▁utilisé▁avant la date à▁laquelle lenregistrement de la▁marque a▁été▁demandé dans▁cet▁État; ou▁tout▁autre▁facteur qui▁permette de▁déterminer si le▁nom commercial▁doit▁être▁considéré▁comme un droit▁antérieur▁existant au▁sens de larticle 16,▁paragraphe 1,▁troisième phrase, de laccord▁ADPIC?","When assessing, in such a case, whether a trade name has a legal basis prior to a trade mark for the purposes of the third sentence of Article 16(1) of the TRIPs Agreement, may it thus be considered as decisive: (i) whether the trade name was well known at least to some extent among the relevant trade circles in the State in which the trade mark is registered and in which protection is sought for it, before the point in time at which registration of the trade mark was applied for in the State in question; or whether the trade name was used in commerce directed to the State in which the trade mark is registered and in which protection is sought for it, before the point in time at which registration of the trade mark was applied for in the State in question; or what other factor may decide whether the trade name is to be regarded as an existing prior right within the meaning of the third sentence of Article 16(1) of the TRIPs Agreement?"
1,"▁Quelles▁sont les▁statistiques▁officielles▁disponibles sur les▁délits en▁matière de▁propriété▁intellectuelle (en particulier,▁quel est le volume de▁produits de▁contrefaçon▁détecté par les▁douanes,▁quels types de▁délits▁sont▁signalés à la police, comment▁ces▁derniers▁sont-ils▁classés,▁quelle est la proportion de▁délits qui font lobjet de▁poursuites par la police et▁sont▁signalés à dautres institutions,▁combien de▁procédures▁judiciaires▁sont▁lancées et▁quels▁sont les▁résultats▁obtenus)?","What official statistics are available on IP crime (ie how much infringing product is detected by Customs, what types and level of IP crime are reported to police, how is this classified, what proportion is acted on by police and referred to other agencies, how many prosecutions are undertaken, and what are the results)?"
2,"Pour▁tirer une conclusion sur ce▁problème, il▁convient de▁répondre aux questions▁suivantes : -▁Alors▁qu▁théorie,▁chaque▁autorité▁nationale de concurrence/régulation▁serait en▁habilitée à▁traiter des▁problèmes de▁tarification transfrontalière à condition▁qu▁ils▁concernent les▁importations et,▁éventuellement, les▁exportations, est-il acceptable que 15▁processus▁décisionnels▁potentiellement▁conflictuels▁traitent au▁même moment▁cette question?","Whilst, in theory, each national regulatory/competition authority might have jurisdiction to deal with cross-border tarification issues insofar as they concern imports and, possibly exports, is it acceptable to have 15 potentially conflicting decision-making processes contemporaneously treating this issue?"
3,"L'article 5 du▁règlement a-t-il▁été▁amendé d'une▁manière non▁conforme aux▁exigences▁procédurales▁visées par l'article 251 CE,▁lors de l'examen du▁projet de▁texte par le▁comité de consultation et, dans l'affirmative, l'article 5 du▁règlement est-il▁invalide et, dans l'affirmative,▁cette▁circonstance (combinée à▁tout▁autre▁élément pertinent)▁affecte-t-elle la▁validité du▁règlement dans son ensemble?","Whether the amendment of Article 5 of the Regulation during consideration of the draft text by the Conciliat

## Tracking

Here comes some details on w&b tracking and the leaderboard to be established...

In [ ]:
import wandb

WANDB_NAME = f'{ds_name}-{model_name}'
GROUP = f'{ds_name}-{model_name}-simple-{lr:.0e}'
NOTES = f'finetuning {model_name} with RAdam lr={lr:.0e}'
CONFIG = {}
TAGS =[model_name, ds_name, 'radam']

In [ ]:
#hide_output
wandb.init(reinit=True, project="fasthugs", entity="fastai_community",
           name=WANDB_NAME, group=GROUP, notes=NOTES, tags=TAGS, config=CONFIG);

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: fastai_community (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.28 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


## Training

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
learn = TransLearner(dls, model, metrics=CorpusBLEUMetric(), loss_func=noop)

In [ ]:
learn.validate()

epoch,train_loss,valid_loss,corpus_bleu,time


(#2) [2.0951087474823,0.31762597662887515]

In [ ]:
learn.fit_one_cycle(2, 1e-4)

epoch,train_loss,valid_loss,corpus_bleu,time
0,1.231185,1.039507,0.655427,06:56
1,1.018177,0.993561,0.674918,06:57


In [ ]:
learn.validate()

(#2) [0.993560791015625,0.6749184356623896]

In [ ]:
df.iloc[10,1]

'Quelle est la province ayant la plus forte densité de population ?'

In [ ]:
inp = tokenizer(df.iloc[10, 1], return_tensors='pt')
pred = learn.generate(inp['input_ids'].to(dls.device))
tokenizer.decode(pred[0].cpu(), skip_special_tokens=True)

'Which province has the highest population density?'